In [ ]:
def build_resnet_block(input_layer, num_cnn=3, channel=64, block_num=1, is_plain=False):
    x = input_layer
    for cnn_num in range(num_cnn):
        identity = x
        x = keras.layers.Conv2D(
            filters=channel,
            kernel_size=(3,3),
            activation='relu',
            kernel_initializer='he_normal',
            padding='same',
            name=f'block{block_num}_conv{cnn_num}'
        )(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Conv2D(
            filters=channel,
            kernel_size=(3,3),
            activation='relu',
            kernel_initializer='he_normal',
            padding='same',
            name=f'block{block_num}_1_conv{cnn_num}'
        )(x)
        if not is_plain:
            identity_channel = identity.shape.as_list()[-1]
            if identity_channel != channel:
                identity = keras.layers.Conv2D(channel, kernel_size=(1, 1), strides=(1, 1), padding="same")(identity)
            x = keras.layers.Add()([x, identity])

    if identity.shape[1] != 1:
        x = keras.layers.MaxPooling2D(
            pool_size=(2, 2),
            strides=2,
            name=f'block{block_num}_pooling'
        )(x)

    return x

def build_resnet(input_shape=(32,32,3), num_classes=10, is_50=True):
    num_cnn_list = [3,4,6,3]
    channel_list = [64,128,256,512]

    input_layer = keras.layers.Input(shape=input_shape)  # input layer를 만들어둡니다.
    output = input_layer

    # Initial conv1 and conv2_x pooling
    output = keras.layers.Conv2D(filters=64, kernel_size=(2,2), strides=2, padding='valid')(output)
    output = keras.layers.BatchNormalization()(output)
    output = keras.layers.MaxPooling2D(pool_size=(2,2), strides=2)(output)

    # Build blocks
    for i, (num_cnn, channel) in enumerate(zip(num_cnn_list, channel_list)):
        output = build_resnet_block(output, num_cnn=num_cnn, channel=channel, block_num=i, is_50=is_50)

    # Final stage
    output = keras.layers.AveragePooling2D(padding='same')(output)
    output = keras.layers.Flatten(name='flatten')(output)
    output = keras.layers.Dense(512, activation='relu', name='fc1')(output)
    output = keras.layers.Dense(num_classes, activation='softmax', name='predictions')(output)

    model = keras.Model(inputs=input_layer, outputs=output)
    return model

In [ ]:
import tensorflow as tf

class conv_block(tf.keras.Model):
    def __init__(self, filters, strides=(2, 2)):
        super(conv_block, self).__init__()

        self.filters1, self.filters2, self.filters3 = filters
        self.strides = strides

        self.conv1 = tf.keras.layers.Conv2D(self.filters1, (1, 1), strides=strides)
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.relu1 = tf.keras.layers.ReLU()

        self.conv2 = tf.keras.layers.Conv2D(self.filters2, (3, 3), strides=(1, 1), padding='same')
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.relu2 = tf.keras.layers.ReLU()

        self.conv3 = tf.keras.layers.Conv2D(self.filters3, (1, 1), strides=(1, 1))
        self.bn3 = tf.keras.layers.BatchNormalization()

        self.shortcut_conv = tf.keras.layers.Conv2D(self.filters3, (1, 1), strides=strides)
        self.shortcut_bn = tf.keras.layers.BatchNormalization()

        self.add = tf.keras.layers.Add()
        self.add_relu = tf.keras.layers.ReLU()

    def call(self, input_tensor, training=False):
        x = self.conv1(input_tensor)
        x = self.bn1(x)
        x = self.relu1(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)

        x = self.conv3(x)
        x = self.bn3(x)

        shortcut = self.shortcut_conv(input_tensor)
        shortcut = self.shortcut_bn(shortcut)

        x = self.add([x, shortcut])
        x = self.add_relu(x)

        return x

class identity_block(tf.keras.Model):
    def __init__(self, filters):
        super(identity_block, self).__init__()

        self.filters1, self.filters2, self.filters3 = filters

        self.conv1 = tf.keras.layers.Conv2D(self.filters1, (1, 1), strides=(1, 1))
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.relu1 = tf.keras.layers.ReLU()

        self.conv2 = tf.keras.layers.Conv2D(self.filters2, (3, 3), strides=(1, 1), padding='same')
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.relu2 = tf.keras.layers.ReLU()

        self.conv3 = tf.keras.layers.Conv2D(self.filters3, (1, 1), strides=(1, 1))
        self.bn3 = tf.keras.layers.BatchNormalization()

        self.add = tf.keras.layers.Add()
        self.add_relu = tf.keras.layers.ReLU()


    def call(self, input_tensor, training=False):
        x = self.conv1(input_tensor)
        x = self.bn1(x)
        x = self.relu1(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)

        x = self.conv3(x)
        x = self.bn3(x)

        x = self.add([x, input_tensor])
        x = self.add_relu(x)

        return x

class ResNet50(tf.keras.Model):
    def __init__(self, nb_classes):
        super(ResNet50, self).__init__()

        self.nb_classes = nb_classes

        # Stage 1 (Conv1 Layer)
        self.zero_padd_1_1 = tf.keras.layers.ZeroPadding2D(padding=(3, 3))
        self.conv_1 = tf.keras.layers.Conv2D(64, (7, 7), strides=(2, 2))
        self.bn_1 = tf.keras.layers.BatchNormalization()
        self.relu_1 = tf.keras.layers.ReLU()
        self.zero_padd_1_2 = tf.keras.layers.ZeroPadding2D(padding=(1, 1))
        self.max_pool = tf.keras.layers.MaxPooling2D((3, 3), strides=(2, 2))

        # Stage 2
        self.stage2 = tf.keras.Sequential()
        self.stage2.add(conv_block([64, 64, 256], strides=(1, 1)))
        self.stage2.add(identity_block([64, 64, 256]))
        self.stage2.add(identity_block([64, 64, 256]))

        # Stage 3
        self.stage3 = tf.keras.Sequential()
        self.stage3.add(conv_block([128, 128, 512]))
        self.stage3.add(identity_block([128, 128, 512]))
        self.stage3.add(identity_block([128, 128, 512]))
        self.stage3.add(identity_block([128, 128, 512]))

        # Stage 4
        self.stage4 = tf.keras.Sequential()
        self.stage4.add(conv_block([256, 256, 1024]))
        self.stage4.add(identity_block([256, 256, 1024]))
        self.stage4.add(identity_block([256, 256, 1024]))
        self.stage4.add(identity_block([256, 256, 1024]))
        self.stage4.add(identity_block([256, 256, 1024]))
        self.stage4.add(identity_block([256, 256, 1024]))

        # Stage 5
        self.stage5 = tf.keras.Sequential()
        self.stage5.add(conv_block([512, 512, 2048]))
        self.stage5.add(identity_block([512, 512, 2048]))
        self.stage5.add(identity_block([512, 512, 2048]))

        self.gap = tf.keras.layers.GlobalAveragePooling2D()
        self.dense = tf.keras.layers.Dense(self.nb_classes, activation='softmax')

    def call(self, input_tensor, training=False):
        x = self.zero_padd_1_1(input_tensor)
        x = self.conv_1(x)
        x = self.bn_1(x)
        x = self.relu_1(x)
        x = self.zero_padd_1_2(x)
        x = self.max_pool(x)

        x = self.stage2(x)
        x = self.stage3(x)
        x = self.stage4(x)
        x = self.stage5(x)

        x = self.gap(x)
        x = self.dense(x)

        return x

model = ResNet50(1000)
model.build((1, 224, 224, 3))
model.summary()

Model: "res_net50"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero_padding2d (ZeroPadding  multiple                 0         
 2D)                                                             
                                                                 
 conv2d (Conv2D)             multiple                  9472      
                                                                 
 batch_normalization (BatchN  multiple                 256       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                multiple                  0         
                                                                 
 zero_padding2d_1 (ZeroPaddi  multiple                 0         
 ng2D)                                                           
                                                         

In [ ]:
# 층을 정의하고 conv과 identity를 for문으로 구현한 방법
from keras import models, layers
from keras import Input
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers, initializers, regularizers, metrics
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import BatchNormalization, Conv2D, Activation, Dense, GlobalAveragePooling2D, MaxPooling2D, ZeroPadding2D, Add

import os
import matplotlib.pyplot as plt
import numpy as np
import math

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_dir = os.path.join('./dataset/1/images/train')
val_dir = os.path.join('./dataset/1/images/val')

train_generator = train_datagen.flow_from_directory(train_dir, batch_size=16, target_size=(224, 224), color_mode='rgb')
val_generator = val_datagen.flow_from_directory(val_dir, batch_size=16, target_size=(224, 224), color_mode='rgb')

# number of classes
K = 4

input_tensor = Input(shape=(224, 224, 3), dtype='float32', name='input')

def conv1_layer(x):
    x = ZeroPadding2D(padding=(3, 3))(x)
    x = Conv2D(64, (7, 7), strides=(2, 2))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = ZeroPadding2D(padding=(1,1))(x)

    return x

def conv2_layer(x):
    x = MaxPooling2D((3, 3), 2)(x)

    shortcut = x

    for i in range(3):
        if (i == 0):
            x = Conv2D(64, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Conv2D(64, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(shortcut)
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)

            x = Add()([x, shortcut])
            x = Activation('relu')(x)

            shortcut = x

        else:
            x = Conv2D(64, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Conv2D(64, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)

            x = Add()([x, shortcut])
            x = Activation('relu')(x)

            shortcut = x

    return x

def conv3_layer(x):
    shortcut = x

    for i in range(4):
        if(i == 0):
            x = Conv2D(128, (1, 1), strides=(2, 2), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut = Conv2D(512, (1, 1), strides=(2, 2), padding='valid')(shortcut)
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)

            x = Add()([x, shortcut])
            x = Activation('relu')(x)

            shortcut = x

        else:
            x = Conv2D(128, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)

            x = Add()([x, shortcut])
            x = Activation('relu')(x)

            shortcut = x

    return x

def conv4_layer(x):
    shortcut = x

    for i in range(6):
        if(i == 0):
            x = Conv2D(256, (1, 1), strides=(2, 2), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Conv2D(256, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Conv2D(1024, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut = Conv2D(1024, (1, 1), strides=(2, 2), padding='valid')(shortcut)
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)

            x = Add()([x, shortcut])
            x = Activation('relu')(x)

            shortcut = x

        else:
            x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Conv2D(256, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Conv2D(1024, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)

            x = Add()([x, shortcut])
            x = Activation('relu')(x)

            shortcut = x

    return x

def conv5_layer(x):
    shortcut = x

    for i in range(3):
        if(i == 0):
            x = Conv2D(512, (1, 1), strides=(2, 2), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Conv2D(512, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Conv2D(2048, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut = Conv2D(2048, (1, 1), strides=(2, 2), padding='valid')(shortcut)
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)

            x = Add()([x, shortcut])
            x = Activation('relu')(x)

            shortcut = x

        else:
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Conv2D(512, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Conv2D(2048, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)

            x = Add()([x, shortcut])
            x = Activation('relu')(x)

            shortcut = x

    return x

x = conv1_layer(input_tensor)
x = conv2_layer(x)
x = conv3_layer(x)
x = conv4_layer(x)
x = conv5_layer(x)

x = GlobalAveragePooling2D()(x)
output_tensor = Dense(K, activation='softmax')(x)

resnet50 = Model(input_tensor, output_tensor)
resnet50.summary()

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Input, MaxPooling2D, BatchNormalization, Activation, Dense, \
GlobalAveragePooling2D,ZeroPadding2D, Add
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50

#resnet = ResNet50()
#resnet.summary()

def resnetConv2D(x, filters=64, strides = (1,1), filters_scale = 1) :
    filters = filters*filters_scale

    x = Conv2D(filters,(1,1),strides=strides,padding='valid')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters, (3, 3), strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(4*filters, (1, 1), strides=(1, 1), padding='valid')(x)

    return x

def resnetConv1(x) :
    x = ZeroPadding2D(padding=(3,3))(x)
    x = Conv2D(64,(7,7),strides=(2,2),padding = 'valid')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = ZeroPadding2D(padding=(1,1))(x)

    return x

def resnetConv2(x) :
    x = MaxPooling2D((3,3), 2)(x)

    sc = x ## shortcut

    for i in range(0,3) :
        if i == 0 :
            x = resnetConv2D(x, strides = (1,1),filters_scale = 1)

            sc = Conv2D(256, kernel_size = (1, 1), strides=(1, 1), padding='valid')(sc)
            x = BatchNormalization()(x)
            sc = BatchNormalization()(sc)

            x = Add()([x, sc])
            x = Activation('relu')(x)

            sc = x

        else :
            x = resnetConv2D(x,strides = (1,1), filters_scale = 1)
            x = BatchNormalization()(x)

            x = Add()([x, sc])
            x = Activation('relu')(x)

            sc = x

    return x

def resnetConv3(x) :
    sc = x

    for i in range(0,4) :
        if i == 0 :
            x = resnetConv2D(x,strides=(2,2),filters_scale=2)
            sc = Conv2D(512,kernel_size =(1,1),strides=(2,2),padding='valid')(sc)
            x=BatchNormalization()(x)
            sc = BatchNormalization()(sc)
            x = Add()([x,sc])
            x = Activation('relu')(x)
            sc = x

        else :
            x = resnetConv2D(x,strides = (1,1), filters_scale=2)
            x = BatchNormalization()(x)

            x = Add()([x,sc])
            x = Activation('relu')(x)

            sc = x

    return x

def resnetConv4(x) :
    sc = x

    for i in range(0, 6) :
        if i == 0 :
            x = resnetConv2D(x, strides = (2,2), filters_scale = 4)

            sc = Conv2D(filters = 1024, kernel_size = (1,1), strides = (2,2), padding = 'valid')(sc)
            x = BatchNormalization()(x)
            sc = BatchNormalization()(sc)
            x = Add()([x, sc])
            Activation('relu')(x)
            sc = x


        else :
            x = resnetConv2D(x, strides = (1,1), filters_scale=4)
            x = BatchNormalization()(x)

            x = Add()([x,sc])
            x = Activation('relu')(x)

            sc = x

    return x

def resnetConv5(x) :
    sc = x

    for i in range(0, 3) :
        if i == 0 :
            x = resnetConv2D(x,strides = (2,2),filters_scale = 8)

            sc = Conv2D(filters = 2048, kernel_size = (1,1), strides = (2,2),padding = 'valid')(sc)
            x = BatchNormalization()(x)
            sc = BatchNormalization()(sc)

            x = Add()([x,sc])
            x = Activation('relu')(x)
            sc = x
        else :
            x = resnetConv2D(x, strides = (1,1),filters_scale = 8)
            x = BatchNormalization()(x)

            x = Add()([x, sc])
            x = Activation('relu')(x)

            sc = x

    return x

def myModel() :
    _input = Input(shape = (128,128,3), dtype = 'float32', name='input')
    x = resnetConv1(_input)
    x = resnetConv2(x)
    x = resnetConv3(x)
    x = resnetConv4(x)
    x = resnetConv5(x)
    x = GlobalAveragePooling2D()(x)

    feature_vector = Dense(256, activation = 'relu',name = 'feature_vector')(x) ## feature vector

    output = Dense(1, activation='relu')(feature_vector)## age

    my_resnet_model = Model(_input,output)

    return my_resnet_model

In [ ]:
def _bn_relu(inputs):
    x = BatchNormalization()(inputs)
    x = Activation('relu')(x)
    return x

def _conv(inputs, filters, kernel_size, strides=1,padding='same',
         kernel_initializer='he_normal', kernel_regularizer=l2(1.e-4)):
    x = Conv2D(filters, kernel_size, strides=strides, padding=padding,
              kernel_initializer=kernel_initializer,
              kernel_regularizer=kernel_regularizer)(inputs)
    return x

def _bn_relu_conv(inputs, filters, kernel_size, strides=1, padding='same',
         kernel_initializer='he_normal', kernel_regularizer=l2(1.e-4)):
    # pre activation
    x = _bn_relu(inputs)
    x = _conv(x, filters, kernel_size, strides=strides, padding=padding,
             kernel_initializer=kernel_initializer,
             kernel_regularizer=kernel_regularizer)
    return x

def _conv_bn_relu(inputs, filters, kernel_size, strides=1, padding='same',
         kernel_initializer='he_normal', kernel_regularizer=l2(1.e-4)):
    x = _conv(inputs, filters, kernel_size, strides=strides, padding=padding,
             kernel_initializer=kernel_initializer,
             kernel_regularizer=kernel_regularizer)
    x = _bn_relu(x)
    return x

def bottleneck(inputs, filters, strides=1):
    x = _conv(inputs, filters, (1,1), strides=strides, padding='valid')
    return x

def conv_block(inputs,filters,rep,is_first=False):

    shortcut = bottleneck(inputs, filters*4, strides=(2 if not is_first else 1))
    shortcut = BatchNormalization()(shortcut)

    x = inputs

    for i in range(rep):
        x = bottleneck(x, filters, strides=(2 if not is_first and i==0 else 1))
        x = _bn_relu_conv(x, filters, (3,3))
        x = _bn_relu_conv(x, filters*4, (1,1), padding='valid')
        x = BatchNormalization()(x)
        x = Add()([x,shortcut])
        x = Activation('relu')(x)

        shortcut = x
    return x

def ResNet50():
    inputs = Input(shape=(224,224,3))
    x = _conv_bn_relu(inputs, 64, (7,7), strides=2, padding='same')
    x = MaxPool2D((3,3), strides=2)(x)

    x = conv_block(x, 64, 3, True)
    x = conv_block(x, 128, 4)
    x = conv_block(x, 256, 6)
    x = conv_block(x, 512, 3)

    x = GlobalAvgPool2D()(x)
    outputs = Dense(num_class, activation='softmax')(x)

    model = Model(inputs, outputs)
    return model

In [ ]:
def identity_block(X, f, filters, training=True, initializer=random_uniform):
    # Retrieve Filters
    F1, F2, F3 = filters

    X_shortcut = X

    # First
    X = Conv2D(filters=F1, kernel_size=1, strides=(1,1), padding='valid')(X)
    X = BatchNormalization(axis = 3)(X, training = training) # Default axis
    X = Activation('relu')(X)

    ## Second
    X = Conv2D(filters=F2, kernel_size=f, strides=(1, 1), padding='same')(X)
    X = BatchNormalization(axis=3)(X, training =training)
    X = Activation('relu')(X)

    # Third
    X = Conv2D(filters=F3, kernel_size=1, strides=(1, 1), padding='valid')(X)
    X = BatchNormalization(axis=3)(X, training=training)

    # Final
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

def convolutional_block(X, f, filters, s = 2, training=True, initializer=glorot_uniform):

    F1, F2, F3 = filters
    X_shortcut = X

    # First
    X = Conv2D(filters=F1, kernel_size=1, strides = (s, s), padding='valid')(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)

    # Second
    X = Conv2D(filters=F2, kernel_size=f, strides=(1, 1), padding='same')(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)

    # Third
    X = Conv2D(filters=F3, kernel_size=1, strides=(1,1), padding='valid')(X)
    X = BatchNormalization(axis=3)(X, training=training)

    X_shortcut = Conv2D(filters=F3, kernel_size=1, strides=(s,s), padding='valid')(X_shortcut)
    X_shortcut = BatchNormalization(axis=3)(X_shortcut, training=training)

    # Final
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

def ResNet50(input_shape = (64, 64, 3), classes = 6):
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], s = 1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])

    # Stage 3
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], s = 2)
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])

    # Stage 4
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], s = 2)
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])

    # Stage 5
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], s = 2)
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])

    # AVGPOOL
    X = AveragePooling2D(pool_size=(2,2))(X)

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax')(X)

    # Create model
    model = Model(inputs = X_input, outputs = X)

    return model